# Ultrasonic Shape Recognition: End-to-End Pipeline

## DeepScan 360: Advanced Ultrasonic Shape Recognition
This notebook documents the end-to-end pipeline for classifying gemometric objects (Triangle, Square, Circle, Hexagon, Oval) using 360° ultrasonic scan data.

**Technical Stack**:
- **Processing**: Signal denoising via Median & Savitzky-Golay filters.
- **Model**: Multi-Scale 1D-Convolutional Neural Network (CNN).
- **Optimization**: Bayesian Hyperparameter Tuning & 5-Fold Cross-Validation.


# **1. Mathematical Foundations of Data Procesing**
Raw ultrasonic data is inherently noisy due to air turbulence, surface reflections, and sensor jitter. Our DataProcessor pipeline applies a sequence of mathematical operations to extract the "true" geometry of the scanned object.

## **A. Median Filtering: Spike ELimination**
The first line of defense is a **Median Filter**. It is exceptionally robust against "impulse noise" (outliers caused by faulty echos).
- **The Math**: A window of size $ n = 3$ slides acros the signal. The center value is replaced by the median of all values within the window.
- **The Benefit**: Unlike a simple mean (average), the median is not skewed by extreme outliers. This perserves the sharp edges of the object while removing random spikes.

## **B. Savitzky-Golay Filter: Smoothing without Distortion**
This is the core of the signal refinement. Standard smoothing often "rounds off" sharp corners, which are vital for distinguishing a triangle from a circle.
- **The Principle**: The filter performs a local polynomial regression (Degree 2) on a sliding window ($L = 11$). It uses the least-sqaures method to find the best fit.
- **The Benefit**: It smooths the noise on flat surfaces but preserves the peaks and valleys (the "corners") much more accurately than a moving average.

## **C. Linear Interpolation: Filling Data Gaps**
Physical outliers (distance $> 15cm$) are dropped, creating `Nan` gaps.
- **The Math**: $y = y_1 + (x - x_1)\frac{y_2-y_1}{x_2-x_1}$
- **The Application**: We connect the points before and after the gap with a straight line. This ensures a continuous signal, which is required for the Neural Network's mathematical operations.

# **2. Neural Network Architecture: Multi-Scale 1D-CNN**
To achieve high accuracy, we utilize a **Multi-Scale Convolutional Architecture**. Instead of a single path, the network looks at the data through different "lenses" simultaneously.

## **A. 1D-Convolutional Layers**
The CNN applies filters to the 163-point scan.
- **Kernel Sizes**: We use kernels like 3, 7 and 15.
    - *Small Kernels (3)*: Detect fine details and sharp corners.
    - *Large Kernels (15)*: Recognize broad geometric patterns and long flat edges.
- **Batch Normalization**: This normalizes the activations of the neurons, significantly speeding up the training and preventing "vanishing gradients".

## **B. Global MaxPooling: Rotational Invariance**
This is a critical step for a rotating scanner.
- **The Operation**: It extracts the maximum value from each feature map across the entire scan.
- **The Logic**: It doesn't matter *where* the corner of the triangle is (at 10° or 190°). The Max-Pooling ensures that the network recognizes the *existance* of the feature regardless of its orientation.

# **3. High-End Optimization Strategy**

## **A. Bayesian Optimization**
Intstead of a random search, we use a Bayesian Oracle. It builds a probability model of the objective function and chooses the next hyperparameters to test based on past results. It balances:
- **Exploration**: Trying new, unknown architecture configurations.
- **Exploitation**: Refining configurations that have already shown high accuracy.

## **B. 5-Fold Cross-Validation**
To ensure our 95%+ accuracy isn't just a "lucky split", we divide the data into 6 parts. The model is trained 5 times, each time using a different part for validation. The final score is the average of all 5 runs, ensuring absolute robustness.
